In [10]:
#Code for dummy disease map. Generates lung mask with random values corresponding to diseases.
import numpy as np

# Define the dimensions of the 3D array
dimensions = (200, 200, 200)

# Define the values and their corresponding probabilities
values = [0, 2, 3, 4]
probabilities = [0.1, 0.7, 0.1, 0.1]

# Create the 3D array with random values based on probabilities
random_values = np.random.choice(values, size=dimensions, p=probabilities)

# Now, 'random_values' is a 3D NumPy array with the specified dimensions and values sampled based on probabilities.

In [11]:
#Function from Nate's code
def update_camera_info(obj, evt):
                    camera = mlab.gcf().scene.camera
                    print("Camera Position:", camera.position)
                    print("Camera Focal Point:", camera.focal_point)
                    print("Camera View Up:", camera.view_up)

In [12]:
#Load in the file
import nrrd
import nibabel as nib
from mayavi import mlab

# Load the NIfTI image using nibabel
file_path = r'Z:\D-Images\SPIROMICS-SubStudy\2-Results-CheckedDoneTemp\1-Done\Case-50-Spiromics-45626090\seg_net\SPI-IA210305-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz'
nii_image = nib.load(file_path)

In [13]:
# Get the image data as a NumPy array
image_data = nii_image.get_fdata().astype(int)
# image_data = np.where(image_data != 0, 1, image_data)
condition = image_data > 0
image_data[condition] = 1

#Create lung pathogen map
dimensions = image_data.shape

# Define the values and their corresponding probabilities
values = [0, 2, 3, 4]
probabilities = [1-0.06, 0.02, 0.02, 0.02]

# Create the 3D array with random values based on probabilities
diseaseEX = np.random.choice(values, size=dimensions, p=probabilities)
diseaseEX = diseaseEX * image_data

#Center of mass coordinates
coords = np.argwhere(image_data)
center_of_mass = np.mean(coords, axis=0)
center_of_mass = (center_of_mass / 10).astype(int)   #Remove /10         #center_of_mass.astype(int) / 10
image_data[*center_of_mass] = 2

In [65]:
import numpy as np

# Define the dimensions of the array
shape = (512, 512, 556)

# Create an empty array filled with zeros
array = np.zeros(shape)

# Define the center and cutoff distance
center = (240, 280, 300)
cutoff_distance = 100

# Create coordinate grids for each dimension
x, y, z = np.meshgrid(
    np.arange(shape[0]),
    np.arange(shape[1]),
    np.arange(shape[2])
)

# Calculate the Euclidean distance from the center for each point
distance = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)

# Apply the Gaussian distribution with a mean of 0 and standard deviation of 1
gaussian = np.exp(-0.5 * (distance / cutoff_distance)**2)

# Threshold the Gaussian values to either 0 or 2
array[gaussian >= 0.9] = 2

# Print a slice of the array to verify
print(array[:, :, 250])  # Print a slice along the z-axis (at z=250)

array = array * image_data

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [64]:
array.max()

0.0

In [66]:
# Create a figure
mlab.figure(bgcolor=(1, 1, 1))  # Set the background color to white

# Visualize the entire image as a volume (adjust opacity as needed)
lungs = mlab.contour3d(image_data[::10,::10,::10], contours=[0.5], opacity = 0.15, color = (128/255, 174/255, 128/255))
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

#Dummy Lung Mask
obj = (array == 2).astype(int)
mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (1, 0, 0))   #Remove later
# obj = (diseaseEX == 3).astype(int)
# mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (0, 0, 1))
# obj = (diseaseEX == 4).astype(int)
# mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (148/255, 0, 211/255))

# Set the camera position and direction to focus on the center of mass
mlab.view(azimuth=90, elevation=90, distance=600/0.546875, focalpoint=center_of_mass)
mlab.gcf().scene.camera.view_angle = 23

#Saving file
outputPath = r"z:/Q- Figures/T32-Poster/Nick/Experiment/"
mlab.savefig(outputPath + "test1.png")   #, magnification=5     <-- Add argument for magnification factor

# Show the visualization
mlab.show()

Exception occurred in traits notification handler for object: <mayavi.tools.modules.IsoSurfaceFactory object at 0x0000025D8002A0C0>, trait: contours, old value: 5, new value: [0.5]
Traceback (most recent call last):
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_notifiers.py", line 342, in __call__
    self.handler(*args)
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\mayavi\tools\modules.py", line 185, in _contours_changed
    self._target.contour.contours = self.contours
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_types.py", line 2695, in validate
    return TraitListObject(self, object, name, value)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_list_object.py", line 584, in __init__
    super().__init__(
  File "

TraitError: Each element of the 'contours' trait of a Contour instance must be 0.0 <= a number <= 0.0, but a value of 0.5 <class 'float'> was specified.

: 

In [9]:
image_data.shape

(512, 512, 556)